In [244]:
import pandas as pd
from datetime import datetime, timedelta, timezone
import numpy as np

In [245]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [246]:
def read_binance_trades(filename:str)->pd.DataFrame:
    """
    Read trade daily files downloaded from binance vision
    """
    df = pd.read_csv(filename,header=None)
    df.columns = ['trade Id','price','qty','quoteQty','time','isBuyerMaker','isBestMatch']
    return df

In [255]:
#Load 20th and 21st of January spot trading data
#Note: The timestamp for SPOT Data from January 1st 2025 onwards will be in microseconds.
df_20 = read_binance_trades('BTCUSDT-trades-2025-01-20.csv')
df_21 = read_binance_trades('BTCUSDT-trades-2025-01-21.csv')
df = pd.concat([df_20,df_21],axis=0).reset_index(drop=True)

In [256]:
#Time window 24hrs after Trump's re-election with timestamps in UTC
trump_reelection_date = datetime(2025, 1, 20, 17, 0,0,tzinfo=timezone.utc)
start_time = trump_reelection_date.timestamp()*1_000_000
end_time = (trump_reelection_date + timedelta(days=1)).timestamp()*1_000_000

In [257]:
#Filter 24hrs window after re-election
df = df[(df["time"] >= start_time) & (df["time"] < end_time)]

In [258]:
#Group by same trade in the same microsecond and on the same side
df_grouped = df.groupby(['time','isBuyerMaker'])[['quoteQty','qty']].sum()

In [266]:
#5 largest trades by USDT
top_5 = df_grouped.reset_index().nlargest(5, "quoteQty",keep='all')

In [267]:
top_5['isBuyerMaker'] = top_5.isBuyerMaker.map({True:'SELL',False:'BUY'})
top_5.rename(columns={'time':'Timestamp (μs)', 'isBuyerMaker':'Side (BUY/SELL)', 'quoteQty':'Amount (USDT)', 'qty':'Amount (BTC)'},inplace=True)

In [268]:
top_5[['Timestamp (μs)','Amount (BTC)','Amount (USDT)','Side (BUY/SELL)']]

,Timestamp (μs),Amount (BTC),Amount (USDT),Side (BUY/SELL)
1362471,1737458674827737,48.720,5048282.731,BUY
434224,1737398772311251,34.361,3578679.637,BUY
199781,1737394752405237,30.242,3086008.255,SELL
1103794,1737437040221884,28.578,2911022.170,SELL
433304,1737398733780866,27.736,2886232.437,BUY
